# Import Libraries

In [ ]:
# connector to bigRED
from dataminer.connector import BigRed,BigRed3
from dataminer.utils import get_secret

# basic data manipulation packages
import pandas as pd
from functools import reduce
import numpy as np

# visualization
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.style as style
import matplotlib.axis as axs
from matplotlib.dates import DateFormatter
%matplotlib inline

import random
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# modeling
from fbprophet import Prophet

# parallelization
import concurrent.futures

import logging
logging.getLogger().setLevel(logging.WARNING)

import matplotlib as mpl

import numpy as np
import os
from datetime import datetime
import pandas as pd
import plotly.express as px

mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False

from sklearn.model_selection import train_test_split

import ast

### Prepare Data

In [ ]:
turnover_df = pd.read_csv('your_data.csv') ### provide your spred sheet with date, target vales
# example calendar_month_end	terminations	average_headcount	turnover
#            YYYY-MM-DD	####	####	%value                add actual data for year, month date, ###, % value 
turnover_process_df = turnover_df[['calendar_month_end','terminations','average_headcount','turnover']]
turnover_process_df['calendar_month_end'] = pd.to_datetime(turnover_process_df['calendar_month_end'])
turnover_process_df

### Process Data

In [ ]:
# descriptive stats
process_merged_df = turnover_process_df
process_merged_df.describe(include='all')

In [ ]:
# Check for any nulls, sort data, converting date time,
process_merged_df = process_merged_df.sort_values(by =['calendar_month_end'],ascending=True)

print(' data Types: ', process_merged_df.info())
max_date = process_merged_df['calendar_month_end'].max().strftime('%Y-%m-%d')
print('Max Calendar Month End date is:', max_date)

print ("Rows     : " ,process_merged_df.shape[0])
print ("Columns  : " ,process_merged_df.shape[1])
print ("\nFeatures : \n" ,process_merged_df.columns.tolist())
print ("\nMissing values :  ", process_merged_df.isnull().any())
print ("\nUnique values :  \n",process_merged_df.nunique())

### Analyze the data

In [ ]:
f, axes = plt.subplots(2, 3, sharex=False, figsize=(40,30))
f.set_figheight(10)
f.set_figwidth(20)
sns.lineplot(ax=axes[0,0],data=process_merged_df, x="calendar_month_end", y="turnover",legend ='full')
axes[0,0].set_title('Turnover')

f.set_figheight(10)
f.set_figwidth(20)
sns.lineplot(ax=axes[0,1],data=process_merged_df, x="calendar_month_end", y="terminations")
axes[0,1].set_title('termination')

f.set_figheight(10)
f.set_figwidth(20)
sns.lineplot(ax=axes[0,2],data=process_merged_df, x="calendar_month_end", y="average_headcount")
axes[0,2].set_title('average_headcount')

f.set_figheight(10)
f.set_figwidth(20)
sns.lineplot(ax=axes[1,0],data=process_merged_df, x="calendar_month_end", y="hires")
axes[1,0].set_title('hires')

f.set_figheight(10)
f.set_figwidth(20)
sns.lineplot(ax=axes[1,1],data=process_merged_df, x="calendar_month_end", y="absents")
axes[1,1].set_title('absents')

f.set_figheight(10)
f.set_figwidth(20)
sns.lineplot(ax=axes[1,2],data=process_merged_df, x="calendar_month_end", y="unemp")
axes[1,2].set_title('unemp')

### Forecast Data

In [ ]:
# Pre setps for forecastin
import os
import sys

class suppress_stdout_stderr(object):
    '''
    A context manager for doing a "deep suppression" of stdout and stderr in
    Python, i.e. will suppress all print, even if the print originates in a
    compiled C/Fortran sub-function.
       This will not suppress raised exceptions, since exceptions are printed
    to stderr just before a script exits, and after the context manager has
    exited (at least, I think that is why it lets exceptions through).
    '''
    def __init__(self):
        # Open a pair of null files
        self.null_fds = [os.open(os.devnull, os.O_RDWR) for x in range(2)]
        # Save the actual stdout (1) and stderr (2) file descriptors.
        self.save_fds = (os.dup(1), os.dup(2))

    def __enter__(self):
        # Assign the null pointers to stdout and stderr.
        os.dup2(self.null_fds[0], 1)
        os.dup2(self.null_fds[1], 2)

    def __exit__(self, *_):
        # Re-assign the real stdout/stderr back to (1) and (2)
        os.dup2(self.save_fds[0], 1)
        os.dup2(self.save_fds[1], 2)
        # Close the null files
        os.close(self.null_fds[0])
        os.close(self.null_fds[1])
        
def MAPE(Y_actual, Y_Predicted):
    
    mape = np.mean(np.abs((Y_actual - Y_Predicted) / Y_actual)) * 100
    return mape

### Turnover Forecast

In [ ]:
ent_m_turnover=process_merged_df[['calendar_month_end','turnover']].rename(columns={'calendar_month_end':'ds','turnover':'y'})[['ds','y']]
ent_m_turnover_model = Prophet()
ent_m_turnover_model.add_country_holidays(country_name='US')
with suppress_stdout_stderr():
    ent_m_turnover_model.fit(ent_m_turnover)
    ent_m_turnover_future_months =ent_m_turnover_model.make_future_dataframe(periods=66,freq='M')
    ent_m_turnover_future_months_forecast =ent_m_turnover_model.predict(ent_m_turnover_future_months)
#ent_m_turnover_future_months_forecast
ent_m_turnover_forecast_all = ent_m_turnover.append(ent_m_turnover_future_months_forecast[ent_m_turnover_future_months_forecast['ds']>max_date][['ds','yhat']].rename(columns={'yhat':'y'}))
ent_m_turnover_actual_forecast_all =ent_m_turnover.merge(ent_m_turnover_future_months_forecast[['ds','yhat','yhat_lower','yhat_upper']],on='ds')
#ent_m_turnover_actual_forecast_all

turnover_mape = MAPE(ent_m_turnover_actual_forecast_all['y'], ent_m_turnover_actual_forecast_all['yhat'])
print('turnover Mape in Percentage:',turnover_mape)

# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
fig = ent_m_turnover_model.plot(ent_m_turnover_future_months_forecast,ax=ax)
plt.title('Actuals vs  Forecast data ')
plt.show()

# Plot the components of the model
fig = ent_m_turnover_model.plot_components(ent_m_turnover_future_months_forecast)

In [ ]:
ent_m_turnover_forecast_all_year =ent_m_turnover_forecast_all.assign(year=lambda y:pd.DatetimeIndex(y['ds']).year,
                                                                                                 month=lambda y:pd.DatetimeIndex(y['ds']).month)
f, axes = plt.subplots(2,figsize=(30,20))
f.set_figheight(15)
f.set_figwidth(20)
sns.barplot(ax=axes[0],data=ent_m_turnover_forecast_all_year, x="month", y="y",hue='year')
axes[0].set_title('Turnover by month for all years')

ent_m_turnover_forecast_all_year_agg = ent_m_turnover_forecast_all_year.groupby(['year']).agg({'y':'sum'}).reset_index()
display(ent_m_turnover_forecast_all_year_agg)

sns.barplot(ax=axes[1],data=ent_m_turnover_forecast_all_year_agg, x="year", y="y")

In [ ]:

print(f" Turnover MAPE is :{turnover_mape} 
")

### Share the data

In [ ]:
#overall_df
alldfs = [var for var in dir() if isinstance(eval(var), pd.core.frame.DataFrame)]
j =[]
forecast = pd.DataFrame()
for i in range(0,len(alldfs)):
    if alldfs[i].endswith('_agg'):
        j.append(alldfs[i])
        
for i in j:
    print(i)

ent_m_turnover_forecast_summary = ent_m_turnover_forecast_all_year_agg.rename(columns={'y':'turnover'})
# ent_m_turnover_forecast_summary = ent_m_turnover_forecast_summary.merge(ent_m_turnover_terminations_forecast_all_year_agg.rename(columns={'y':'terminations'}), on='year')
# ent_m_turnover_forecast_summary = ent_m_turnover_forecast_summary.merge(ent_m_turnover_average_headcount_forecast_all_year_agg.rename(columns={'y':'avrage_headcount'}), on='year')
# ent_m_turnover_forecast_summary = ent_m_turnover_forecast_summary.merge(ent_m_turnover_unemp_forecast_all_year_agg.rename(columns={'y':'unemp'}), on='year')
# ent_m_turnover_forecast_summary = ent_m_turnover_forecast_summary.merge(ent_m_turnover_absents_forecast_all_year_agg.rename(columns={'y':'absents'}), on='year')
# ent_m_turnover_forecast_summary = ent_m_turnover_forecast_summary.merge(ent_m_turnover_hires_forecast_all_year_agg.rename(columns={'y':'hires'}), on='year')

# ent_m_turnover_forecast_summary = ent_m_turnover_forecast_summary.merge(ent_m_turnover_absents_unemp_forecast_all_year_agg.rename(columns={'y':'absents_unemp'}), on='year')
# ent_m_turnover_forecast_summary = ent_m_turnover_forecast_summary.merge(ent_m_turnover_absents_hires_forecast_all_year_agg.rename(columns={'y':'absents_hires'}), on='year')
# ent_m_turnover_forecast_summary = ent_m_turnover_forecast_summary.merge(ent_m_turnover_unemp_hires_forecast_all_year_agg.rename(columns={'y':'unemp_hires'}), on='year')
# #ent_m_turnover_forecast_summary = ent_m_turnover_forecast_summary.merge(ent_m_turnover_terminations_average_headcount_forecast_all_year_agg.rename(columns={'y':'terminations_average_headcount'}), on='year')

# ent_m_turnover_forecast_summary = ent_m_turnover_forecast_summary.merge(ent_m_turnover_absents_unemp_hires_forecast_all_year_agg.rename(columns={'y':'absents_unemp_hires'}), on='year')


display(ent_m_turnover_forecast_summary)
ent_m_turnover_forecast_summary_2021_plus =ent_m_turnover_forecast_summary[ent_m_turnover_forecast_summary['year'] >2019]
ent_m_turnover_forecast_summary_2021_plus.plot(x="year", y=["turnover","terminations","avrage_headcount", 
                                                  ], kind="bar",figsize=(20,10))